In [ ]:
import csv
import codecs
from tqdm import tqdm
from urllib.request import urlretrieve

In [ ]:
rows = []
with codecs.open('../data/HomologyTAPE/pdb_chain_scop_uniprot.tsv', encoding='utf-8') as f:
    for row in csv.DictReader(f, skipinitialspace=True, delimiter='\t'):
        rows.append(row)
print(len(rows))
print(rows[0])

In [ ]:
fold_dic = {}
for row in rows:
    fold_dic[row['SCOP_ID']] = {}
    fold_dic[row['SCOP_ID']]['PDB-chain'] = row['PDB'].upper() + '-' + row['CHAIN']
    fold_dic[row['SCOP_ID']]['uniprot'] = row['SP_PRIMARY']

In [ ]:
fold_dic['d101ma_']

In [ ]:
fold_all = []

In [ ]:
with open('../data/HomologyTAPE/test_fold.txt') as f:
    test_file_lines = f.readlines()
test_lines = {}
test_pdb = {}
failed_test_scop = []
for line in test_file_lines:
    data = line.split('\t')
    data = [d.strip() for d in data]
    test_lines[data[0]] = data[-1]
    fold_all.append(data[0])
    try:
        if fold_dic[data[0]]['PDB-chain'] not in test_pdb.keys():
            test_pdb[fold_dic[data[0]]['PDB-chain']] = []
        test_pdb[fold_dic[data[0]]['PDB-chain']].append(data[-1])
    except:
        failed_test_scop.append(data[0])

In [ ]:
with open('../data/HomologyTAPE/training.txt') as f:
    train_file_lines = f.readlines()
train_lines = {}
train_pdb = {}
failed_train_scop = []
for line in train_file_lines:
    data = line.split('\t')
    data = [d.strip() for d in data]
    train_lines[data[0]] = data[-1]
    fold_all.append(data[0])
    try:
        if fold_dic[data[0]]['PDB-chain'] not in train_pdb.keys():
            train_pdb[fold_dic[data[0]]['PDB-chain']] = []
        train_pdb[fold_dic[data[0]]['PDB-chain']].append(data[-1])
    except:
        failed_train_scop.append(data[0])

In [ ]:
with open('../data/HomologyTAPE/validation.txt') as f:
    val_file_lines = f.readlines()
val_lines = {}
val_pdb = {}
failed_val_scop = []
for line in val_file_lines:
    data = line.split('\t')
    data = [d.strip() for d in data]
    val_lines[data[0]] = data[-1]
    fold_all.append(data[0])
    try:
        if fold_dic[data[0]]['PDB-chain'] not in val_pdb.keys():
            val_pdb[fold_dic[data[0]]['PDB-chain']] = []
        val_pdb[fold_dic[data[0]]['PDB-chain']].append(data[-1])
    except:
        failed_val_scop.append(data[0])

In [ ]:
fold_dic['d1ysya1']

In [ ]:
for key, value in test_pdb.items():
    test_pdb[key] = list(set(value))
for key, value in train_pdb.items():
    train_pdb[key] = list(set(value))
for key, value in val_pdb.items():
    val_pdb[key] = list(set(value))

In [ ]:
print(len(test_file_lines))
print(len(train_file_lines))
print(len(val_file_lines))
print(len(failed_test_scop))
print(len(failed_train_scop))
print(len(failed_val_scop))

In [ ]:
print(len(test_pdb))
print(len(train_pdb))
print(len(val_pdb))
print(len(test_pdb))

In [ ]:
all_pdb = list(test_pdb.keys()) + list(train_pdb.keys()) + list(val_pdb.keys())

In [ ]:
all_pdb = [pdb[:4] for pdb in all_pdb]
print(len(all_pdb))

In [ ]:
import json
no_download = []
with open('../data/HomologyTAPE/all_download.json') as f:
    download_pdb = json.load(f)
with open('../data/HomologyTAPE/download_pdb.json') as f:
    download_fold = json.load(f)

In [ ]:
download_fold = [i.upper() for i in download_fold]

In [ ]:
for pdb in tqdm(all_pdb):
    if pdb not in download_pdb:
        no_download.append(pdb)

In [ ]:
pdb_chains = list(test_pdb.keys()) + list(train_pdb.keys()) + list(val_pdb.keys())

In [ ]:
with open('../data/EnzymeCommission/nrPDB-EC_annot_copy.tsv') as f:
    ec_lines = f.readlines()
with open('../data/GeneOntology/nrPDB-GO_annot.tsv') as f:
    go_lines = f.readlines()
download_chains = []
download_ec_chains = [ec.split('\t')[0] for ec in ec_lines[1:]]
download_go_chains = [ec.split('\t')[0] for ec in go_lines[12:]]
download_chains = download_ec_chains + download_go_chains

In [ ]:
import os
download_fold = os.listdir('../data/HomologyTAPE/all/')
download_fold = [i.split('.')[0] for i in download_fold]

In [ ]:
download_fold[0]

In [ ]:
download_chains = list(set(download_chains))
need_download = []
for pdb in tqdm(pdb_chains):
    if pdb not in download_chains and pdb[:4] not in download_pdb and pdb[:4] not in download_fold:
        need_download.append(pdb)

In [ ]:
print(need_download)

In [ ]:
need_download = [pdb[:4].lower() for pdb in need_download]

In [ ]:
len(list(set(need_download)))

In [ ]:
import os
downloaded = os.listdir('../data/HomologyTAPE/all')
downloaded = [pdb[:4].lower() for pdb in downloaded]

In [ ]:
len(list(set(downloaded)))

In [ ]:
n_d = []
for pdb in need_download:
    if pdb not in downloaded:
        n_d.append(pdb)

In [ ]:
n_d = list(set(n_d))
print(len(n_d))

In [ ]:
n_d = []
for scop in fold_all:
    if scop[-1] != '_':
        n_d.append(scop)

In [ ]:
len(n_d)
n_d = list(set(n_d))

In [27]:
failed = 0
root_dir = '../data/HomologyTAPE/'
idx = 0
download_pdb = []
failed_pdb = []
for scopid in tqdm(n_d):
    
    # if idx == 10:
    #     break
    try:
        
        url = 'https://scop.berkeley.edu/downloads/pdbstyle/pdbstyle-1.75/'+scopid[2:4] + '/'+ scopid + '.ent'
        # print(url)
        urlretrieve(url, root_dir +'all/' + scopid + '.pdb')
        download_pdb.append(scopid.lower())
    except:
        print("cannot download:", scopid)
        failed += 1
        failed_pdb.append(scopid.lower())
    idx += 1
with open('../data/HomologyTAPE/download_scopid.json', 'w') as f:
    json.dump(download_pdb, f)
with open('../data/HomologyTAPE/failed_scopid.json', 'w') as f:
    json.dump(failed_pdb, f)

  0%|          | 1/8113 [00:24<54:17:24, 24.09s/it]

cannot download: d2h3la1


In [ ]:
import json
failed = 0
root_dir = '../data/HomologyTAPE/'
idx = 0
# download_pdb = []
# failed_pdb = []
with open('../data/HomologyTAPE/failed_scopid.json', 'r') as f:
    failed_pdb = json.load(f)
for scopid in tqdm(n_d):
    
    # if idx == 10:
    #     break
    try:
        
        url = 'https://scop.berkeley.edu/downloads/pdbstyle/pdbstyle-1.75/'+scopid[2:4] + '/'+ scopid + '.ent'
        # print(url)
        urlretrieve(url, root_dir +'all/' + scopid + '.pdb')
        download_pdb.append(scopid.lower())
    except:
        print("cannot download:", scopid)
        failed += 1
        failed_pdb.append(scopid.lower())
    idx += 1
with open('../data/HomologyTAPE/download_scopid_1.json', 'w') as f:
    json.dump(download_pdb, f)
with open('../data/HomologyTAPE/failed_scopid_1.json', 'w') as f:
    json.dump(failed_pdb, f)

In [ ]:
import h5py

In [ ]:
f = h5py.File("../data/HomologyTAPE/d2gyco1.hdf5","r")
print(f.keys())

In [ ]:
f['amino_chains'][()]